# 2024/06/05 LODES User Manual
_Author: Meaghan Freund_

Queries and uses of the LODES ADRIO Template

Set up for the LODES call, users are able to change the call depending on which states or further granularities they specifically want to  see

**CHANGE FOR GEOSCOPES PENDING**

In [ ]:
from pathlib import Path

import numpy as np

from epymorph.data_shape import Shapes
from epymorph.geo.spec import DynamicGeoSpec, Year, attrib
from epymorph.geography.us_census import (BlockGroupScope, CountyScope,
                                          StateScope, TractScope)

spec = DynamicGeoSpec(
    attributes=[
        attrib('label', str, Shapes.N),
        attrib('commuters', int, Shapes.NxN),
        attrib('commuters_29_under', int, Shapes.NxN),
        attrib('commuters_30_to_54', int, Shapes.NxN),
        attrib('commuters_55_over', int, Shapes.NxN),
        attrib('commuters_1250_under_earnings', int, Shapes.NxN),
        attrib('commuters_1251_to_3333_earnings', int, Shapes.NxN),
        attrib('commuters_3333_over_earnings', int, Shapes.NxN),
        attrib('commuters_goods_producing_industry', int, Shapes.NxN),
        attrib('commuters_trade_transport_utility_industry', int, Shapes.NxN),
        attrib('commuters_3333_over_earnings', int, Shapes.NxN),
        attrib('all_jobs', int, Shapes.NxN),
        attrib('primary_jobs', int, Shapes.NxN),
        attrib('all_private_jobs', int, Shapes.NxN),
        attrib('private_primary_jobs', int, Shapes.NxN),
        attrib('all_federal_jobs', int, Shapes.NxN),
        attrib('federal_primary_jobs', int, Shapes.NxN)
    ],
    time_period=Year(2015),
    scope=CountyScope.in_states_by_code(["AZ", "NM", "CO", "UT"]),
    source={
        'label': 'LODES:name',
        'home_geoid': 'LODES',
        'work_geoid': 'LODES',
        'commuters': 'LODES',
        'commuters_29_under': 'LODES',
        'commuters_30_to_54': 'LODES',
        'commuters_55_over': 'LODES',
        'commuters_1250_under_earnings': 'LODES',
        'commuters_1251_to_3333_earnings': 'LODES',
        'commuters_3333_over_earnings': 'LODES',
        'commuters_goods_producing_industry': 'LODES',
        'commuters_trade_transport_utility_industry': 'LODES',
        'commuters_other_industry': 'LODES',
        'all_jobs': 'LODES',
        'primary_jobs': 'LODES',
        'all_private_jobs': 'LODES',
        'private_primary_jobs': 'LODES',
        'all_federal_jobs': 'LODES',
        'federal_primary_jobs': 'LODES'
    }
)

In [ ]:
from epymorph.geo.adrio import adrio_maker_library
from epymorph.geo.dynamic import DynamicGeo

geo = DynamicGeo.from_library(spec, adrio_maker_library)

# Query Types

## Label
The first type of query outputs the gathered input to better represent how the matrix is being read and output.

In [ ]:
values = [geo['label']]

print(values)

## Total Commuters

The first matrix query is the "commuters" call, which outputs a matrix with the given locations and granularities as the positions with the total number of commuters to and from each location.

In [ ]:
values = [geo['commuters']]

print(values)

## Query Workers by Categories

Further beyond total commuters, LODES holds data about the type of workers that commute from location to location, being categorized under their age, monthly earnings, and the industry they work under.

## Commuters By Age

LODES holds information about commuting employees age through three age ranges, being:
- Workers 29 years old and younger
- Workers 30 years old to 54 years old
- Workers 55 years old and older

The three ranges can be called as separate queries:

### 29 and Younger

In [ ]:
values = [geo['commuters_29_under']]

print(values)

### 30 to 54 Years Old

In [ ]:
values = [geo['commuters_30_to_54']]

print(values)

### 55 and Older

In [ ]:
values = [geo['commuters_55_over']]

print(values)

## Commuters by Earnings

LODES datasets specify the earnings by monthly salaries, similarly to the age categories with the three different ranges. 

- Workers earning $1250 or under per month
- Workers earning $1250+ to $3333 per month
- Workers earning $3333+ per month

With the same manner as the ages, these three categories can be called as three separate queries:

### $1250 and Under Earned Monthly

In [ ]:
values = [geo['commuters_1250_under_earnings']]

print(values)

### Between $1250.01 and $3333 Earned Monthly

In [ ]:
values = [geo['commuters_1251_to_3333_earnings']]

print(values)

### Above $3333 Earned Monthly

In [ ]:
values = [geo['commuters_3333_over_earnings']]

print(values)

## Commuters By Industry

LODES also tracks the basic industry that commuters work under. Similar to age and earnings, LODES separates the industry into three categories, however, the categories are not split by ranges as industry is not as cut and dry as earnings or age.

LODES splits commuters by the following industries:
- Workers that work under the goods and producing industry
- Workers that work under trade, transport, or utility industries
- Workers that work under any industry other than the above claimed industries


### Goods and Producing Industry

In [ ]:
values = [geo['commuters_goods_producing_industry']]

print(values)

### Trade, Transport, Utility Industries

In [ ]:
values = [geo['commuters_trade_transport_utility_industry']]

print(values)

### Any Other Industry

In [ ]:
values = [geo['commuters_other_industry']]

print(values)